In [2]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop Token:")

In [3]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [16]:
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith('.java'):
            print(f"Processing file: {file}") # This line prints the file's name
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                docs.extend(loader.load_and_split())
            except Exception as e:
                pass


Processing file: EtherNetIP.java
Processing file: Main.java
Processing file: Tag.java
Processing file: TagList.java
Processing file: TagListener.java
Processing file: CipException.java
Processing file: ConnectionData.java
Processing file: DeviceInfo.java
Processing file: EthernetLink.java
Processing file: Identity.java
Processing file: InterfaceConfiguration.java
Processing file: InterfaceFlags.java
Processing file: TcpIpInterface.java
Processing file: CIPMultiRequestProtocol.java
Processing file: CIPReadDataProtocol.java
Processing file: CIPWriteDataProtocol.java
Processing file: Connection.java
Processing file: ConnectionDataProtocol.java
Processing file: Encapsulation.java
Processing file: ForwardCloseProtocol.java
Processing file: ForwardOpenProtocol.java
Processing file: GetAttributeAllProtocol.java
Processing file: GetAttributeListProtocol.java
Processing file: GetAttributeSingle.java
Processing file: GetConnectionDataProtocol.java
Processing file: GetEthernetLinkProtocol.java
Pr

In [17]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 1575, which is longer than the specified 1000
Created a chunk of size 1340, which is longer than the specified 1000
Created a chunk of size 1413, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of size 2058, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1154, which is longer than the specified 1000
Created a chunk of size 2180, which is longer than the specified 1000
Created a chunk of size 1002, which is longer than the specified 1000
Created a chunk of size 1308, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 2172, which is longer than the specified 1000
Created a chunk of size 1906, which is longer than the specified 1000
Created a chunk of size 1123, which is longer than the specified 1000
Created a chunk of s

In [19]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("CIP.pdf")
pagesCIP = loader.load_and_split()

In [7]:
loader = PyPDFLoader("ENIP.pdf")
pagesENIP = loader.load_and_split()

In [8]:
loader = PyPDFLoader("Logix5000.pdf")
pagesLogix = loader.load_and_split()

In [9]:
mergedContents=texts+pagesCIP+pagesENIP+pagesLogix

In [10]:
username = "zhan4442"  # replace with your username from app.activeloop.ai
db = DeepLake(
    dataset_path=f"hub://{username}/CodeTextLinker",
    embedding_function=embeddings,
)
db.add_documents(mergedContents)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


\

Dataset(path='hub://zhan4442/CodeTextLinker', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
 embedding  embedding  (1921, 1536)  float32   None   
    id        text      (1921, 1)      str     None   
 metadata     json      (1921, 1)      str     None   
   text       text      (1921, 1)      str     None   


['df358c58-3cde-11ee-b60b-4851c55e5357',
 'df358c59-3cde-11ee-98a7-4851c55e5357',
 'df358c5a-3cde-11ee-b082-4851c55e5357',
 'df358c5b-3cde-11ee-b8f9-4851c55e5357',
 'df358c5c-3cde-11ee-9aea-4851c55e5357',
 'df358c5d-3cde-11ee-8297-4851c55e5357',
 'df358c5e-3cde-11ee-9092-4851c55e5357',
 'df358c5f-3cde-11ee-bdca-4851c55e5357',
 'df358c60-3cde-11ee-a0b8-4851c55e5357',
 'df358c61-3cde-11ee-9f19-4851c55e5357',
 'df358c62-3cde-11ee-99ea-4851c55e5357',
 'df358c63-3cde-11ee-bac4-4851c55e5357',
 'df358c64-3cde-11ee-b229-4851c55e5357',
 'df358c65-3cde-11ee-b51c-4851c55e5357',
 'df358c66-3cde-11ee-9369-4851c55e5357',
 'df358c67-3cde-11ee-bdf0-4851c55e5357',
 'df358c68-3cde-11ee-9e80-4851c55e5357',
 'df358c69-3cde-11ee-96ea-4851c55e5357',
 'df358c6a-3cde-11ee-a556-4851c55e5357',
 'df358c6b-3cde-11ee-b910-4851c55e5357',
 'df358c6c-3cde-11ee-ba1e-4851c55e5357',
 'df358c6d-3cde-11ee-8dfa-4851c55e5357',
 'df358c6e-3cde-11ee-a04c-4851c55e5357',
 'df358c6f-3cde-11ee-88e9-4851c55e5357',
 'df358c70-3cde-

In [12]:
db = DeepLake(
    dataset_path="hub://zhan4442/CodeTextLinker",
    read_only=True,
    embedding_function=embeddings,
)

Deep Lake Dataset in hub://zhan4442/CodeTextLinker already exists, loading from the storage


In [13]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-4")  # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [25]:
chat_history = []

In [28]:
questions = [
    # "For the Java code you have, I write them to try to decode the traffic of Allen Bradley PLC,\
    # from CIP level to ENIP level. Like in the main class, I start with the choice of TCP/UCP \
    # then I dig into encapsulation part by using encapsulation.decode(buffer, responseSize, log). \
    # In the Encapsulation class, which belongs to package etherip.protocol, I dig inside other protocols \
    # like listIdentityProtocol by using if(command_code ==0x0004), do you understand my way of decoding?",
    "When you meet a structure like else if(command_code ==0x0004), you'll know it's time to go inside the next layer. I try to go inside layer by layer. Please list all layers following Encapsulation in the code.",
]

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: When you meet a structure like else if(command_code ==0x0004), you'll know it's time to go inside the next layer. I try to go inside layer by layer. Please list all layers following Encapsulation in the code. 

**Answer**: The following list shows the next layers following the Encapsulation layer based on the given context:

1. ListServicesProtocol Layer: This layer is invoked when the command code is 0x0004.

2. ListIdentitiesProtocol Layer: This layer is invoked when the command code is 0x0063.

3. ListInterfacesProtocol Layer: This layer is invoked when the command code is 0x0064.

4. RegisterSessionProtocol Layer: This layer is invoked when the command code is 0x0065.

These layers are based on the command codes listed in the context. Each command code leads to a different layer of protocol processing. 

